In [59]:
import pandas as pd
import psycopg2
import requests
import io
import pandas_gbq
from google.cloud import storage
def download_csv_from_gcs(bucket_name, file_name,project_name)->pd.DataFrame:
    """
    Downloads a CSV file from Google Cloud Storage and returns it as a DataFrame.

    Args:
        bucket_name (str): Name of the Google Cloud Storage bucket.
        file_name (str): Name of the CSV file to be downloaded.
        project_name (str): Name of the project in google cloud.

    Returns:
        pd.DataFrame: DataFrame containing the data from the downloaded CSV file.
    """
    storage_client = storage.Client(project=project_name)
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)
    file_string = blob.download_as_text()
    df = pd.read_csv(io.StringIO(file_string),sep=';',low_memory=False)
    return df




In [71]:
df_mineria_resultado_campanas_csv=download_csv_from_gcs('customer_experience_bucket', 'mineria_resultado_campanas.csv','customer experience')


c:\Users\jpmva\.virtualenvs\ETL---customer-experience-project-c9QD_7vZ\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [60]:
df_mineria_ventas_csv=download_csv_from_gcs('customer_experience_bucket', 'mineria_venta.csv','customer experience')


c:\Users\jpmva\.virtualenvs\ETL---customer-experience-project-c9QD_7vZ\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [54]:

dtype={
'ID FACTURA':'str',
'FECHA_LARGA':'str',
'ID_PDV':'int',
'ID CLIENTE':'int',
'NUMERO FACTURA':'str',
'UNIDADES':'str',
'PORCENTAJE DESCUENTO':'str',
'PRECIO FULL':'int',
'porcentaje_iva':'int',
'PRECIO ANTES IVA':'int',
'PRECIO CON IVA':'int',
'FORMAS DE PAGO':'str',
'ID REFERENCIA':'str',
'PROMOCION':'str',
'HABEAS DATA VENTA':'str',
'VALOR DESCUENTO':'str',
'COSTO TOTAL':'str',
'COSTO UNITARIO':'int',
'ID_VENDEDOR':'int',
'ESTADO ID VENTA':'str',
'ESTADO CELULAR VENTA':'str',
'ESTADO EMAIL VENTA':'str',
'FECHA':'str',
'CONSECUTIVO TRX':'str',
'CAPTURA DE DATOS CEL':'str',
'CAPTURA DE DATOS CEL+CORREO':'str',
'CAPTURA DE DATOS COMPLETO':'str',
'RANGO TICKET':'str',
'TICKET':'float'}
df_mineria_ventas_csv=download_csv_from_gcs('customer_experience_bucket', 'mineria_venta.csv','customer experience')

c:\Users\jpmva\.virtualenvs\ETL---customer-experience-project-c9QD_7vZ\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [62]:
old_column_names=df_mineria_ventas_csv.columns.to_list()
new_column_names=['id_factura',
'fecha_larga',
'id_pdv',
'id_cliente',
'numero_factura',
'unidades',
'porcentaje_descuento',
'precio_full',
'porcentaje_iva',
'precio_antes_iva',
'precio_con_iva',
'formas_de_pago',
'id_referencia',
'promocion',
'habeas_data_venta',
'valor_descuento',
'costo_total',
'costo_unitario',
'id_vendedor',
'estado_id_venta',
'estado_celular_venta',
'estado_email_venta',
'fecha',
'consecutivo_trx',
'captura_de_datos_cel',
'captura_de_datos_celcorreo',
'captura_de_datos_completo',
'rango_ticket',
'ticket']
cambio_cols=dict(zip(old_column_names,new_column_names))
df_mineria_ventas_csv.rename(columns=cambio_cols,inplace=True)

In [63]:
df_mineria_ventas_csv['ticket']=df_mineria_ventas_csv.ticket.astype('int')
df_mineria_ventas_csv['precio_antes_iva']=df_mineria_ventas_csv['precio_antes_iva'].str.replace('$', '').str.replace(' ', '').str.replace('.','').astype('int')

In [66]:
df_mineria_ventas_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048327 entries, 0 to 1048326
Data columns (total 29 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   id_factura                  1048327 non-null  object 
 1   fecha_larga                 1048327 non-null  object 
 2   id_pdv                      1048327 non-null  int64  
 3   id_cliente                  1048327 non-null  int64  
 4   numero_factura              1048327 non-null  object 
 5   unidades                    1048320 non-null  float64
 6   porcentaje_descuento        1024637 non-null  object 
 7   precio_full                 1048327 non-null  object 
 8   porcentaje_iva              1048327 non-null  object 
 9   precio_antes_iva            1048327 non-null  object 
 10  precio_con_iva              1048327 non-null  object 
 11  formas_de_pago              880401 non-null   object 
 12  id_referencia               1048327 non-null  int64  
 1

In [70]:
df_mineria_ventas_csv.groupby('rango_ticket')['ticket'].min()

rango_ticket
< 70.000               -100179
>=200.000 < 500.000     200009
>=500.000              1000622
>=70.000 < 200.000      100000
Name: ticket, dtype: object

In [73]:
from google.cloud import bigquery


project_id='customer-experience-384423'
client = bigquery.Client(project=project_id)
table_id_mineria_ventas_csv = 'customer-experience-384423.Data_cruda.mineria_venta'
table_id = "mineria_venta"
dataset_id = "Data_cruda"
# Create the BigQuery table object
table = bigquery.Table(f"{project_id}.{dataset_id}.{table_id}")
job = client.load_table_from_dataframe(df_mineria_ventas_csv.astype('string'), table)
job.result()


c:\Users\jpmva\.virtualenvs\ETL---customer-experience-project-c9QD_7vZ\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


KeyboardInterrupt: 

In [72]:
old_column_names=df_mineria_resultado_campanas_csv.columns.to_list()
new_column_names=['id_cliente',
'key_movimiento',
'codigo_campana',
'codigo_campana_master',
'estado',
'nombre_campana',
'fecha_inicio',
'fecha_fin',
'campo_fecha',
'variable_1',
'detalle_variable_1',
'filtro_general_1',
'variable_2',
'detalle_variable_2',
'filtro_general_2',
'variable_3',
'detalle_variable_3',
'filtro_general_3',
'copy',
'cliente_contactado',
'tipo_contacto',
'rango_compra_historica',
'rango_recencia',
'rfm',
'rfm_segmento',
'campana_efectiva_tipo',
'mantener_campana']
    
cambio_cols=dict(zip(old_column_names,new_column_names))
df_mineria_resultado_campanas_csv.rename(columns=cambio_cols,inplace=True)

In [76]:
project_id='customer-experience-384423'
table_id_mineria_resultado_campanas_csv = 'customer-experience-384423.Data_cruda.mineria_resultado_campanas'
pandas_gbq.to_gbq(df_mineria_resultado_campanas_csv.astype('string'), table_id_mineria_resultado_campanas_csv, project_id=project_id, if_exists='replace')